## Select articles: look for hateful comments

In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
from mongoengine import connect
from hatespeech_models import Tweet, Article

client = connect("hatespeech-selection")
db = client["hatespeech-selection"]

In [108]:
Article.objects(dummy__ne=True).count()

74550

In [109]:
seeds = [    
    "China", 
    "Cuba", 
    "cubano", 
    "bolivia",
    "Trump", 
    "judío", 
    "camionero", 
    "agresor", 
    #"policía",
    "\"presos\"",
    "ladrón", 
    "reprimir", 
    "represión",
    #"juez", 
    #"justicia", 
    "penal", 
    "criminal", 
    "delito", "mamá", 
    "\"de género\"", 
    "\"aborto\"", 
    "\"actriz\"", 
    "feminista",
    "piqueteros", 
    "villa", 
    "monos",
    "\"Pichot\"",
    "femicidio",
    #"cristina", 
    #"macri", 
    "\"Evo Morales\"", 
    "\"canosa\"", 
    "\"movimientos sociales\"",
    "\"protesta\"",
    "\"médica\"",
    "\"Florencia Kirchner\"",
    "\"enfermera\"",
    "\"madre\"",
    "\"organizaciones sociales\"",
    "\"villa\"",
    "\"Nati Jota\"",
    "\"Ernestina Pais\"",
    "\"Mica Viciconte\"",
    "\"Funes Ugarte\"",
    "\"la modelo\"",
    "\"la actriz\"",
    "\"la periodista\"",
    "\"Carla\"",
    "\"la conductora\"",
    "\"la cantante\"",
    "\"Cande\"",
    "\"personal doméstico\"",
    "\"empleadas\"",
]

seed_ids = set()
seed_articles = []

queries = {}

for word in seeds:
    query = Article.objects(dummy__ne=True, comments__19__exists=True).search_text(word)
    
    nuevos = 0
    queries[word] = query
    
    for art in query:
        if art.id not in seed_ids:
            nuevos += 1
            seed_ids.add(art.id)
            seed_articles.append(art)
    print(f"{word:<30} ---> {len(seed_articles):<5}, {nuevos:<4} nuevos")
    
len(seed_articles)


China                          ---> 966  , 966  nuevos
Cuba                           ---> 1120 , 154  nuevos
cubano                         ---> 1167 , 47   nuevos
bolivia                        ---> 1307 , 140  nuevos
Trump                          ---> 1491 , 184  nuevos
judío                          ---> 1516 , 25   nuevos
camionero                      ---> 1623 , 107  nuevos
agresor                        ---> 1685 , 62   nuevos
"presos"                       ---> 2107 , 422  nuevos
ladrón                         ---> 2250 , 143  nuevos
reprimir                       ---> 2332 , 82   nuevos
represión                      ---> 2365 , 33   nuevos
penal                          ---> 2783 , 418  nuevos
criminal                       ---> 2843 , 60   nuevos
delito                         ---> 3100 , 257  nuevos
mamá                           ---> 3398 , 298  nuevos
"de género"                    ---> 3522 , 124  nuevos
"aborto"                       ---> 3566 , 44   nuevos
"actriz"  

5737

In [110]:
from hatespeech_models import Reply

comment_seeds = [
    "judío",
    "sionista",
    "\"terrorista\"",
    "monos",
    "\"soros\"",
    "\"puto\"",
    "travesti",
    "\"trolo\"",
    "maricon",
    "\"degenerado\"",
    "\"trabuco\"",
    "picaporte",
]

new_article_ids = set()

for word in comment_seeds:
    nuevos = 0
    
    for rep in Reply.objects.search_text(word):
        if len(rep.article.comments) < 20:
            continue
        
        art = rep.article
        
        if art.id not in seed_ids:
            nuevos += 1
            seed_ids.add(art.id)
            seed_articles.append(art)
    
    print(f"{word:<30} ---> {len(seed_articles):<5}, {nuevos:<4} nuevos")


judío                          ---> 5795 , 58   nuevos
sionista                       ---> 5817 , 22   nuevos
"terrorista"                   ---> 5997 , 180  nuevos
monos                          ---> 6245 , 248  nuevos
"soros"                        ---> 6321 , 76   nuevos
"puto"                         ---> 6574 , 253  nuevos
travesti                       ---> 6578 , 4    nuevos
"trolo"                        ---> 6625 , 47   nuevos
maricon                        ---> 6634 , 9    nuevos
"degenerado"                   ---> 6663 , 29   nuevos
"trabuco"                      ---> 6663 , 0    nuevos
picaporte                      ---> 6669 , 6    nuevos


In [111]:
distinct_titles = set(art.title for art in seed_articles)

len(distinct_titles)

5585

In [112]:
len(seed_ids)

6669

In [114]:
for rep in Reply.objects.search_text("\"terrorista\""):
    print(rep.text)

@clarincom tu imagen e ineficiencia aterran,terrorista sanitario.
@clarincom Convertir a tu propia hija en Terrorista para luego mandarla a morir, trae aparejado este resentimiento con la vida y la gente. 👆🏻👆🏻👆🏻
@LANACION Y QUIÉN CARAJO ES ESTA BOSTA DE MUJER QUE LE ARRUINÓ LA VIDA A UNA FAMILIA PARA VENIR A DECIRNOS A LA SOCIEDAD QUE PODEMOS O NO 
 ESCUCHAR?? MEJOR Q SE VAYA A LAVAR EL OGT Q LO TIENE MUY SUCIO ESTA TERRORISTA MAL PARIDA!!
@infobae Tendria que caberle sanciones a los periodistas terroristas y manipuladores
@clarincom Como Fujimori en Peru en los 2000 persiguendo terroristas por las calles ( un vende humo muy peligroso este muchacho) como le gusta las camaras
@infobae Si estuviera viviendo en Argentina ya me hubiese vuelto terrorista y estaría viendo de comprar armas y bombas. Cómo son tan HDP.
@clarincom Se imaginan al enano  Chiche y sus terroristas sanitarios manejando CABA ?
@LANACION Nunca tantos muertos como los que mataron los asesinos, terroristas de sus hijos.E

In [99]:
for i, art in enumerate(Article.objects(dummy__ne=True, comments__19__exists=True).search_text("Agustina")):
    print((f"{'=' * 80 }" + '\n') * 3)
    print(f"{i+1} -- {art}")
    print("\n\n")
    #print(art.body[:600])


1 -- 1291894486939967489 - infobae (1112 comentarios)
Oscar González Oro se fue a vivir a Uruguay: “Recuperé mi libertad”




2 -- 1277921778636111873 - infobae (257 comentarios)
El Gobierno prepara una profunda reforma militar para limitar el accionar de las Fuerzas Armadas




3 -- 1274027052471947269 - LANACION (258 comentarios)
La mamá de Enrique Sacco de 88 años tiene coronavirus




4 -- 1292030891062108167 - infobae (38 comentarios)
Oscar González Oro se fue a vivir a Uruguay: “Recuperé mi libertad”




5 -- 1301010446695239680 - infobae (25 comentarios)
El final de Pippo: cerró el clásico restaurante de la gastronomía porteña tras 83 años de historia




6 -- 1258082309053583360 - infobae (148 comentarios)
Alberto Fernández: “Salir de la cuarentena como reclaman algunos es llevar a la muerte a miles de argentinos”




7 -- 1299126296186966017 - infobae (69 comentarios)
Eduardo Duhalde atribuyó sus dichos sobre un posible golpe de Estado a un comportamiento “psicótico momentáne

6482

## Cuál es la intersección con los que tienen odio?

Busquemos el grupo odioso y veamos cuántas están acá


In [90]:
from groups.models import Group

Group.objects.no_dereference()

[Comments 0.15 group with 4429 tweets
    , Comments 0.20 group with 2502 tweets
    , Comments 0.25 group with 1397 tweets
    , Comments 0.30 group with 744 tweets
    ]

Veamos con el de 0.30

In [91]:
group = Group.objects[3]

In [92]:
group.articles[0].tweet_id

1228308352607846401

In [93]:
in_seeds = sum(art.id in seed_ids for art in group.articles)

print(f"De las notas odiosas, está el {100 * (in_seeds / len(group.articles)):.2f}%")

De las notas odiosas, está el 79.70%


¿Cuáles faltan?

In [94]:
missing = [art for art in group.articles if art.id not in seed_ids]



In [95]:
for i, art in enumerate(missing):
    print((f"{'=' * 80 }" + '\n') * 3)
    print(f"{i+1} -- {art}")
    print("\n\n")
    #print(art.body[:600])


1 -- 1232042149618708480 - LANACION (51 comentarios)
Coronavirus: Ginés González García informó que van a controlar a los pasajeros de vuelos provenientes de Italia




2 -- 1232777369951195136 - LANACION (24 comentarios)
¿Por qué genera tanto pánico el Coronavirus?




3 -- 1234243785015431168 - LANACION (32 comentarios)
Instruyen a la policía bonaerense para actuar en caso de coronavirus




4 -- 1234908165705125888 - LANACION (20 comentarios)
Coronavirus: la clínica Suizo Argentina atiende al paciente que volvió de Italia el domingo




5 -- 1235582505232687104 - clarincom (29 comentarios)
Un turista alemán bajo estudio por coronavirus se fue del hospital: “No hay ninguna ley que me pueda retener aquí”




6 -- 1238864603955597312 - LANACION (20 comentarios)
Coronavirus en la Argentina: el primer vuelo de repatriación será el miércoles y desde Miami




7 -- 1239664127880003584 - LANACION (22 comentarios)
Volvió de EE.UU., violó la cuarentena, la denunció su padre y ahora tiene cus

In [49]:
missing[60].comments

[(0.97) @LANACION ¿Por qué mejor no los trasladan a Inglaterra?,
 (0.00) @LANACION Argentina, el aguantadero de america latina.,
 (0.00) @LANACION Que sea a cambio del canal Beagle,
 (0.00) @LANACION Mmm no se si es viable esto...,
 (0.01) @LANACION Están locos???,
 (0.00) @LANACION Y bueno... Si somos el hazmerreír del mundo!!! Lógico,
 (0.17) @LANACION a Malvinas mandenlos....ahi los quieren mucho,
 (0.12) @LANACION Se nos cagan de risa , somos el ajuste social de este vecindario basurero,
 (0.12) @LANACION Vamo.... A pedir ayuda a la reina....a los piratas....acá no mijo!,
 (1.00) @LANACION mmmm, malvinas.. mmm, no creo che,
 (0.00) @LANACION Ahí tienen a su tan amado modelo chileno.,
 (0.03) @LANACION que vivos,
 (0.16) @LANACION Pero están locos,
 (0.99) @LANACION Che, y si le pasamos las coordenadas de Shile a los Ingleses, como los Shilenos le pasaron las nuestras, para que los ayuden como en la guerra de Malvinas, claro lo digo de onda, sin rencor,
 (0.07) @LANACION Ah bueno...